In [1]:
import ipeadatapy as ipea
import pandas as pd

df = ipea.list_series()

In [8]:
df

df_filtrado = df[df['NAME'].str.contains('arrecadação', case=False, na=False)]

df_filtrado


,CODE,NAME
657,IPTUCH,Arrecadação do IPTU incluindo receitas da dívi...
658,ISSCH,Arrecadação do ISS incluindo receitas da dívid...
659,ITBICH,Arrecadação do ITBI incluindo receitas da dívi...
660,MPAS12_ARRBT12,Previdência Social - fluxo de caixa da previdê...
661,MPAS12_ARRLIQ12,Previdência Social - fluxo de caixa da previdê...
670,RECPROPCH,Arrecadação total dos impostos municipais (IPT...
703,SRF12_TOTGER12,Arrecadação das receitas federais - receita bruta


In [3]:
df_arrecadacao_federal_rb = ipea.timeseries('SRF12_TOTGER12')

df_arrecadacao_federal_rb_limpo = df_arrecadacao_federal_rb.rename(columns={
    'CODE': 'code',
    'RAW DATE': 'raw_date',
    'DAY': 'day',
    'MONTH': 'month',
    'YEAR': 'year',
    'VALUE (R$)': 'real_value'
})


df_arrecadacao_federal_rb_limpo.columns

df_arrecadacao_federal_rb_limpo['real_value'] = df_arrecadacao_federal_rb_limpo['real_value'] * 1_000_000 # multiplicando pra ficar no numero bruto

df_arrecadacao_federal_rb_limpo['real_value'] = df_arrecadacao_federal_rb_limpo['real_value'].round(2)

df_arrecadacao_federal_rb_limpo



,code,raw_date,day,month,year,real_value
DATE,,,,,,
1993-01-01,SRF12_TOTGER12,1993-01-01T00:00:00-02:00,1,1,1993,3.059528e+07
1993-02-01,SRF12_TOTGER12,1993-02-01T00:00:00-02:00,1,2,1993,3.465490e+07
1993-03-01,SRF12_TOTGER12,1993-03-01T00:00:00-03:00,1,3,1993,4.158396e+07
1993-04-01,SRF12_TOTGER12,1993-04-01T00:00:00-03:00,1,4,1993,5.819186e+07
1993-05-01,SRF12_TOTGER12,1993-05-01T00:00:00-03:00,1,5,1993,8.717628e+07
...,...,...,...,...,...,...
2024-08-01,SRF12_TOTGER12,2024-08-01T00:00:00-03:00,1,8,2024,2.016221e+11
2024-09-01,SRF12_TOTGER12,2024-09-01T00:00:00-03:00,1,9,2024,2.031687e+11
2024-10-01,SRF12_TOTGER12,2024-10-01T00:00:00-03:00,1,10,2024,2.479203e+11


In [4]:
arquivo = 'https://seculoxx.ibge.gov.br/images/seculoxx/economia/financas_publicas/fp01_receitadespesauniao.xls'

df_receita = pd.read_excel(arquivo)

df_receita = df_receita.rename(columns={'Receita e despesa da União': 'Ano', 
                                        'Unnamed: 1': 'Receita Arrecadada', 
                                        'Unnamed: 2': 'Despesa realizada', 
                                        'Unnamed: 3': 'Superavit(+) ou Deficit(-)'})

df_receita['Ano'] = pd.to_numeric(df_receita['Ano'], errors='coerce')

df_receita['Ano'] = df_receita['Ano'].fillna(0).astype(int)


df_filtrado = df_receita[(df_receita['Ano'] >= 1980) & (df_receita['Ano'] <= 1993)]

def map_moeda(ano):
    if 1970 <= ano <= 1985:
        return "Milhares de cruzeiros (Cr$ 1.000)"
    elif 1986 <= ano <= 1988:
        return "Milhares de cruzados (Cz$ 1.000)"
    elif ano == 1989:
        return "Milhares de cruzados novos (NCz$ 1.000)"
    elif 1990 <= ano <= 1992:
        return "Milhares de cruzeiros (Cr$ 1.000)"
    elif ano == 1993:
        return "Milhares de cruzeiros reais (CR$ 1.000)"
    else:
        return "Moeda não definida"
    
df_filtrado['moeda'] = df_filtrado['Ano'].apply(map_moeda)


df_filtrado

C:\Users\Dogec\AppData\Local\Temp\ipykernel_17244\513175933.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['moeda'] = df_filtrado['Ano'].apply(map_moeda)


,Ano,Receita Arrecadada,Despesa realizada,Superavit(+) ou Deficit(-),moeda
87,1980,1230018000,1190994000,39024000,Milhares de cruzeiros (Cr$ 1.000)
88,1981,2351966000,2254895000,97071000,Milhares de cruzeiros (Cr$ 1.000)
89,1982,4774815000,4619772000,155043000,Milhares de cruzeiros (Cr$ 1.000)
90,1983,11779847000,11104585000,675262000,Milhares de cruzeiros (Cr$ 1.000)
91,1984,36414053000,33817215000,2596838000,Milhares de cruzeiros (Cr$ 1.000)
92,1985,134851122000,130425844000,4425278000,Milhares de cruzeiros (Cr$ 1.000)
94,1986,562828690,548108711,14719979,Milhares de cruzados (Cz$ 1.000)
95,1987,1692405901,1615099596,77306305,Milhares de cruzados (Cz$ 1.000)
96,1988,15949586311,15857926137,91660174,Milhares de cruzados (Cz$ 1.000)
98,1989,547059356,577265809,-30206453,Milhares de cruzados novos (NCz$ 1.000)
